In [1]:
!pip install pyradiomics
import yaml
from radiomics import featureextractor
import pandas as pd
import os

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 48.2 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Discarding https://files.pythonhosted.org/packages/03/c1/20fc2c50ab1e3304da36d866042a1905a2b05a1431ece35448ab6b4578f2/pyradiomics-3.1.0.tar.gz (from https://pypi.org/simple/pyradiomics/): Requested pyradiomics from https://files.pythonhosted.org/packages/03/c1/20fc2c50ab1e3304da36d866042a1905a2b05a1431ece35448ab6b4578f2/pyradiomics-3.1.0.tar.gz has inconsistent version: expected '3.1.0', but metadata has '3.0.1a1'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 50.4 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.1/118.1 kB 4.7 MB/s eta 0:00:00
  Created wheel for pyradiomics: filename=pyradiomics-3.0.1-cp311-cp311-linux_x86_64.whl size=169811 sh

In [3]:

# Gốc dataset
base_dir = "/kaggle/input/t1-mri-scan/Cirrhosis_T1_3D"

# Cấu trúc folder
splits = {
    "train": {"images": "train_images", "masks": "train_masks"},
    "test": {"images": "test_images", "masks": "test_masks"},
    "valid": {"images": "valid_images", "masks": "valid_masks"}
}

rows = []

for split, dirs in splits.items():
    img_dir = os.path.join(base_dir, dirs["images"])
    mask_dir = os.path.join(base_dir, dirs["masks"])

    # duyệt qua toàn bộ file trong folder ảnh
    for f in os.listdir(img_dir):
        if f.endswith(".nii.gz") or f.endswith(".nii"):  # nhận cả .nii và .nii.gz
            patient_id = os.path.splitext(os.path.splitext(f)[0])[0]
            img_path = os.path.join(img_dir, f)
            mask_path = os.path.join(mask_dir, f)

            if os.path.exists(mask_path):
                rows.append({
                    "PatientID": patient_id,
                    "Image": img_path,
                    "Mask": mask_path,
                    "Split": split
                })
            else:
                print(f"⚠️ Không tìm thấy mask cho {f} ({split})")

# Xuất CSV
df = pd.DataFrame(rows)
df.to_csv("dataset_1.csv", index=False)

print(f"✅ Tạo file dataset.csv với {len(df)} dòng")


✅ Tạo file dataset.csv với 310 dòng


In [4]:

# Gốc dataset
base_dir = "/kaggle/input/t2-mri-scan/Cirrhosis_T2_3D"

# Cấu trúc folder
splits = {
    "train": {"images": "train_images", "masks": "train_masks"},
    "test": {"images": "test_images", "masks": "test_masks"},
    "valid": {"images": "valid_images", "masks": "valid_masks"}
}

rows = []

for split, dirs in splits.items():
    img_dir = os.path.join(base_dir, dirs["images"])
    mask_dir = os.path.join(base_dir, dirs["masks"])

    # duyệt qua toàn bộ file trong folder ảnh
    for f in os.listdir(img_dir):
        if f.endswith(".nii.gz") or f.endswith(".nii"):  # nhận cả .nii và .nii.gz
            patient_id = os.path.splitext(os.path.splitext(f)[0])[0]
            img_path = os.path.join(img_dir, f)
            mask_path = os.path.join(mask_dir, f)

            if os.path.exists(mask_path):
                rows.append({
                    "PatientID": patient_id,
                    "Image": img_path,
                    "Mask": mask_path,
                    "Split": split
                })
            else:
                print(f"⚠️ Không tìm thấy mask cho {f} ({split})")

# Xuất CSV
df = pd.DataFrame(rows)
df.to_csv("dataset_2.csv", index=False)

print(f"✅ Tạo file dataset.csv với {len(df)} dòng")


✅ Tạo file dataset.csv với 318 dòng


In [ ]:
config_dict = {
    "imageType": {
        "Original": {},
        "Wavelet": {},
        "LoG": {"sigma": [1.0, 2.0, 3.0]}  # Lọc Gaussian Laplace
    },
    "featureClass": {
        "shape": None,
        "firstorder": None,
        "glcm": None,
        "glrlm": None,
        "glszm": None,
        "gldm": None,
        "ngtdm": None
    },
    "setting": {
        "resampledPixelSpacing": [1, 1, 1],  # Chuẩn hóa voxel spacing
        "interpolator": "sitkBSpline",
        "binWidth": 25,         # Quantization bin width
        "padDistance": 5,
        "correctMask": True
    },
}
# Lưu config
with open("config_mri.yaml", "w") as f:
    yaml.dump(config_dict, f, default_flow_style=False)
extractor = featureextractor.RadiomicsFeatureExtractor("config_mri.yaml")
df = pd.read_csv("dataset_1.csv")
all_features = []
for idx, row in df.iterrows():
    try:
        features = extractor.execute(row["Image"], row["Mask"])
        # Giữ lại các feature bắt đầu bằng "original"
        feature_dict = {k: v for k, v in features.items() if k.startswith("original")}
        # Thêm thông tin ID, Split
        feature_dict["PatientID"] = row["PatientID"]
        feature_dict["Split"] = row["Split"]
        all_features.append(feature_dict)
        print(f"✅ Done {row['PatientID']} ({row['Split']})")
    except Exception as e:
        print(f"❌ Error {row['PatientID']}: {e}")
features_df = pd.DataFrame(all_features)
features_df.to_csv("features_mri_1.csv", index=False)
print("🎉 Features saved to features_mri_1.csv")

In [ ]:
config_dict = {
    "imageType": {
        "Original": {},
        "Wavelet": {},
        "LoG": {"sigma": [1.0, 2.0, 3.0]}  # Lọc Gaussian Laplace
    },
    "featureClass": {
        "shape": None,
        "firstorder": None,
        "glcm": None,
        "glrlm": None,
        "glszm": None,
        "gldm": None,
        "ngtdm": None
    },
    "setting": {
        "resampledPixelSpacing": [1, 1, 1],  # Chuẩn hóa voxel spacing
        "interpolator": "sitkBSpline",
        "binWidth": 25,         # Quantization bin width
        "padDistance": 5,
        "correctMask": True
    },
}
# Lưu config
with open("config_mri.yaml", "w") as f:
    yaml.dump(config_dict, f, default_flow_style=False)
extractor = featureextractor.RadiomicsFeatureExtractor("config_mri.yaml")
df = pd.read_csv("dataset_2.csv")
all_features = []
for idx, row in df.iterrows():
    try:
        features = extractor.execute(row["Image"], row["Mask"])
        # Giữ lại các feature bắt đầu bằng "original"
        feature_dict = {k: v for k, v in features.items() if k.startswith("original")}
        # Thêm thông tin ID, Split
        feature_dict["PatientID"] = row["PatientID"]
        feature_dict["Split"] = row["Split"]
        all_features.append(feature_dict)
        print(f"✅ Done {row['PatientID']} ({row['Split']})")
    except Exception as e:
        print(f"❌ Error {row['PatientID']}: {e}")
features_df = pd.DataFrame(all_features)
features_df.to_csv("features_mri_2.csv", index=False)
print("🎉 Features saved to features_mri_2.csv")

In [5]:
import os
import pandas as pd

# Thư mục gốc dataset
base_dir = "/kaggle/input/healthy/Healthy_subjects/T1_W_Healthy"

# Định nghĩa thư mục T1 images và T1 masks
img_dir = os.path.join(base_dir, "T1_images")
mask_dir = os.path.join(base_dir, "T1_masks")

rows = []

# Duyệt toàn bộ ảnh trong folder T1_images
for f in os.listdir(img_dir):
    if f.endswith(".nii.gz") or f.endswith(".nii"):
        patient_id = os.path.splitext(os.path.splitext(f)[0])[0]
        img_path = os.path.join(img_dir, f)
        mask_path = os.path.join(mask_dir, f)

        if os.path.exists(mask_path):
            rows.append({
                "PatientID": patient_id,
                "Image": img_path,
                "Mask": mask_path
            })
        else:
            print(f"⚠️ Không tìm thấy mask cho {f}")

# Xuất CSV
df = pd.DataFrame(rows)
df.to_csv("dataset_T1_healthy.csv", index=False)

print(f"✅ Tạo file dataset_T1.csv với {len(df)} dòng")


✅ Tạo file dataset_T1.csv với 55 dòng


In [6]:
import os
import pandas as pd

# Thư mục gốc dataset
base_dir = "/kaggle/input/healthy/Healthy_subjects/T2_W_Healthy"

# Định nghĩa thư mục T1 images và T1 masks
img_dir = os.path.join(base_dir, "T2_images")
mask_dir = os.path.join(base_dir, "T2_masks")

rows = []

# Duyệt toàn bộ ảnh trong folder T1_images
for f in os.listdir(img_dir):
    if f.endswith(".nii.gz") or f.endswith(".nii"):
        patient_id = os.path.splitext(os.path.splitext(f)[0])[0]
        img_path = os.path.join(img_dir, f)
        mask_path = os.path.join(mask_dir, f)

        if os.path.exists(mask_path):
            rows.append({
                "PatientID": patient_id,
                "Image": img_path,
                "Mask": mask_path
            })
        else:
            print(f"⚠️ Không tìm thấy mask cho {f}")

# Xuất CSV
df = pd.DataFrame(rows)
df.to_csv("dataset_T2_healthy.csv", index=False)

print(f"✅ Tạo file dataset_T2.csv với {len(df)} dòng")


✅ Tạo file dataset_T2.csv với 55 dòng


In [ ]:
config_dict = {
    "imageType": {
        "Original": {},
        "Wavelet": {},
        "LoG": {"sigma": [1.0, 2.0, 3.0]}  # Lọc Gaussian Laplace
    },
    "featureClass": {
        "shape": None,
        "firstorder": None,
        "glcm": None,
        "glrlm": None,
        "glszm": None,
        "gldm": None,
        "ngtdm": None
    },
    "setting": {
        "resampledPixelSpacing": [1, 1, 1],  # Chuẩn hóa voxel spacing
        "interpolator": "sitkBSpline",
        "binWidth": 25,         # Quantization bin width
        "padDistance": 5,
        "correctMask": True
    },
}
# Lưu config
with open("config_mri.yaml", "w") as f:
    yaml.dump(config_dict, f, default_flow_style=False)
extractor = featureextractor.RadiomicsFeatureExtractor("config_mri.yaml")
df = pd.read_csv("dataset_T1_healthy.csv")
all_features = []
for idx, row in df.iterrows():
    try:
        features = extractor.execute(row["Image"], row["Mask"])
        # Giữ lại các feature bắt đầu bằng "original"
        feature_dict = {k: v for k, v in features.items() if k.startswith("original")}
        # Thêm thông tin ID, Split
        feature_dict["PatientID"] = row["PatientID"]
        all_features.append(feature_dict)
        print(f"✅ Done {row['PatientID']}")
    except Exception as e:
        print(f"❌ Error {row['PatientID']}: {e}")
features_df = pd.DataFrame(all_features)
features_df.to_csv("features_mri_healthy_T1.csv", index=False)
print("🎉 Features saved to features_mri_healthy.csv")

In [ ]:
config_dict = {
    "imageType": {
        "Original": {},
        "Wavelet": {},
        "LoG": {"sigma": [1.0, 2.0, 3.0]}  # Lọc Gaussian Laplace
    },
    "featureClass": {
        "shape": None,
        "firstorder": None,
        "glcm": None,
        "glrlm": None,
        "glszm": None,
        "gldm": None,
        "ngtdm": None
    },
    "setting": {
        "resampledPixelSpacing": [1, 1, 1],  # Chuẩn hóa voxel spacing
        "interpolator": "sitkBSpline",
        "binWidth": 25,         # Quantization bin width
        "padDistance": 5,
        "correctMask": True
    },
}
# Lưu config
with open("config_mri.yaml", "w") as f:
    yaml.dump(config_dict, f, default_flow_style=False)
extractor = featureextractor.RadiomicsFeatureExtractor("config_mri.yaml")
df = pd.read_csv("dataset_T2_healthy.csv")
all_features = []
for idx, row in df.iterrows():
    try:
        features = extractor.execute(row["Image"], row["Mask"])
        # Giữ lại các feature bắt đầu bằng "original"
        feature_dict = {k: v for k, v in features.items() if k.startswith("original")}
        # Thêm thông tin ID, Split
        feature_dict["PatientID"] = row["PatientID"]
        all_features.append(feature_dict)
        print(f"✅ Done {row['PatientID']}")
    except Exception as e:
        print(f"❌ Error {row['PatientID']}: {e}")
features_df = pd.DataFrame(all_features)
features_df.to_csv("features_mri_healthy_T2.csv", index=False)
print("🎉 Features saved to features_mri_healthy.csv")

In [ ]:
# Nén folder working thành zip
!zip -r results.zip /kaggle/working/

# Tạo link tải
from IPython.display import FileLink
FileLink("results.zip")


In [ ]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

# Nếu GPU OK sẽ in: Device: cuda


In [ ]:
import os
import torch
import torchvision.transforms as T
from torchvision.models import resnet50
from PIL import Image
import pandas as pd
from tqdm import tqdm


# ---------------------------
# 1. RESNET50 PRETRAINED
# ---------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"

model = resnet50(weights="IMAGENET1K_V1")
model = torch.nn.Sequential(*list(model.children())[:-1])  # bỏ FC → embedding 2048D
model.to(device)
model.eval()

# ---------------------------
# 2. IMAGE TRANSFORM
# ---------------------------
transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    ),
])


# ---------------------------
# 3. EXTRACT FEATURE FROM 1 IMAGE
# ---------------------------
def extract_deep_feature(img_path):
    img = Image.open(img_path).convert("RGB")
    x = transform(img).unsqueeze(0).to(device)
    with torch.no_grad():
        feat = model(x).squeeze().cpu().numpy()
    return feat


# ---------------------------
# 4. MAIN EXTRACTION LOOP
# ---------------------------
def extract_from_dataset(base_dir,x):

    rows = []
    splits = ["train", "valid", "test"]

    for split in splits:
        img_root = os.path.join(base_dir, f"{split}_images")

        if not os.path.exists(img_root):
            print(f"❌ Folder không tồn tại: {img_root}")
            continue

        # mỗi folder bên trong = 1 bệnh nhân
        patients = [p for p in os.listdir(img_root) 
                    if os.path.isdir(os.path.join(img_root, p))]

        for patient_id in tqdm(patients, desc=f"Split {split}"):

            patient_folder = os.path.join(img_root, patient_id)

            image_files = [
                f for f in os.listdir(patient_folder)
                if f.lower().endswith((".png", ".jpg", ".jpeg"))
            ]

            for img_file in image_files:

                img_path = os.path.join(patient_folder, img_file)

                feat = extract_deep_feature(img_path)

                rows.append({
                    "Split": split,
                    "PatientID": patient_id,
                    "ImageFile": img_file,
                    **{f"F{i}": feat[i] for i in range(len(feat))}
                })

    df = pd.DataFrame(rows)
    df.to_csv(f"deep_features_2d_{x}.csv", index=False)
    print(f"✅ Saved deep_features_2d_{x}.csv")

    return df


# RUN:
df1 = extract_from_dataset("/kaggle/input/2dpicture/Cirrhosis_T1_3D",1)
df2 = extract_from_dataset("/kaggle/input/2dpicture/Cirrhosis_T2_3D",2)


In [ ]:
import pandas as pd
import numpy as np

# Load file deep feature của bạn
# Lấy danh sách cột feature F0..F2047
feature_cols = [c for c in df1.columns if c.startswith("F")]

# Group theo bệnh nhân
patient_df = (
    df1.groupby(["Split", "PatientID"])[feature_cols]
      .mean()     # mean pooling
      .reset_index()
)

# Lưu file
patient_df.to_csv("deep_features_patient_T1.csv", index=False)

print("✅ Saved deep_features_patient_T1.csv")
print("Số bệnh nhân:", len(patient_df))


In [ ]:
import pandas as pd
import numpy as np

# Load file deep feature của bạn
df=df2
# Lấy danh sách cột feature F0..F2047
feature_cols = [c for c in df.columns if c.startswith("F")]

# Group theo bệnh nhân
patient_df = (
    df.groupby(["Split", "PatientID"])[feature_cols]
      .mean()     # mean pooling
      .reset_index()
)

# Lưu file
patient_df.to_csv("deep_features_patient_T2.csv", index=False)

print("✅ Saved deep_features_patient_T2.csv")
print("Số bệnh nhân:", len(patient_df))


In [ ]:
import os
import torch
import torchvision.transforms as T
from torchvision.models import resnet50
from PIL import Image
import pandas as pd
from tqdm import tqdm


# ---------------------------
# 1. RESNET50 PRETRAINED
# ---------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"

model = resnet50(weights="IMAGENET1K_V1")
model = torch.nn.Sequential(*list(model.children())[:-1])  # bỏ FC → embedding 2048D
model.to(device)
model.eval()

# ---------------------------
# 2. IMAGE TRANSFORM
# ---------------------------
transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    ),
])


# ---------------------------
# 3. EXTRACT FEATURE FROM 1 IMAGE
# ---------------------------
def extract_deep_feature(img_path):
    img = Image.open(img_path).convert("RGB")
    x = transform(img).unsqueeze(0).to(device)
    with torch.no_grad():
        feat = model(x).squeeze().cpu().numpy()
    return feat


# ---------------------------
# 4. MAIN EXTRACTION LOOP
# ---------------------------
def extract_from_dataset(base_dir,x):

    rows = []
    splits = ["T1","T2"]

    for split in splits:
        img_root = os.path.join(base_dir, f"{split}_images")

        if not os.path.exists(img_root):
            print(f"❌ Folder không tồn tại: {img_root}")
            continue

        # mỗi folder bên trong = 1 bệnh nhân
        patients = [p for p in os.listdir(img_root) 
                    if os.path.isdir(os.path.join(img_root, p))]

        for patient_id in tqdm(patients, desc=f"Split {split}"):

            patient_folder = os.path.join(img_root, patient_id)

            image_files = [
                f for f in os.listdir(patient_folder)
                if f.lower().endswith((".png", ".jpg", ".jpeg"))
            ]

            for img_file in image_files:

                img_path = os.path.join(patient_folder, img_file)

                feat = extract_deep_feature(img_path)

                rows.append({
                    "Split": split,
                    "PatientID": patient_id,
                    "ImageFile": img_file,
                    **{f"F{i}": feat[i] for i in range(len(feat))}
                })

    df = pd.DataFrame(rows)
    df.to_csv(f"deep_features_2d_{x}.csv", index=False)
    print(f"✅ Saved deep_features_2d_{x}.csv")

    return df


# RUN:
df1 = extract_from_dataset("/kaggle/input/healthy-data/Healthy_subjects/T1_W_Healthy",11)
df2 = extract_from_dataset("/kaggle/input/healthy-data/Healthy_subjects/T2_W_Healthy",12)


In [ ]:
import pandas as pd
import numpy as np

# Load file deep feature của bạn
# Lấy danh sách cột feature F0..F2047
feature_cols = [c for c in df1.columns if c.startswith("F")]

# Group theo bệnh nhân
patient_df = (
    df1.groupby(["Split", "PatientID"])[feature_cols]
      .mean()     # mean pooling
      .reset_index()
)

# Lưu file
patient_df.to_csv("deep_features_patient_T1_healthy.csv", index=False)

print("✅ Saved deep_features_patient_T1.csv")
print("Số bệnh nhân:", len(patient_df))


In [ ]:
import pandas as pd
import numpy as np

# Load file deep feature của bạn
df=df2
# Lấy danh sách cột feature F0..F2047
feature_cols = [c for c in df.columns if c.startswith("F")]

# Group theo bệnh nhân
patient_df = (
    df.groupby(["Split", "PatientID"])[feature_cols]
      .mean()     # mean pooling
      .reset_index()
)

# Lưu file
patient_df.to_csv("deep_features_patient_T2_healthy.csv", index=False)

print("✅ Saved deep_features_patient_T2.csv")
print("Số bệnh nhân:", len(patient_df))


In [ ]:
# Nén folder working thành zip
!zip -r results.zip /kaggle/working/

# Tạo link tải
from IPython.display import FileLink
FileLink("results.zip")
